In [159]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import vstack, sqrt
import pandas as pd
import networkx as nx
from torch.autograd import Variable
import torch, time, sys,math
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score,mean_tweedie_deviance
from torch import Tensor,nn
from torch.nn import Linear,Sigmoid,ReLU, Module
from torch.optim import SGD
from torch.nn import MSELoss,CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from Data_Utils import *
from Plot_Utils import *
from Math_Utils import *
torch.autograd.set_detect_anomaly(True)
#LSTM(sequence_len,batchsize,dim)

def sigmoid(x):
    return 1/(1+np.exp(-x))
def IdentityActivator(x):
    return x

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,)
        self.output_layer = nn.Linear(in_features=hidden_size, out_features=output_size)

    def forward(self, x, h_state):
        # x (batch, time_step, input_size)
        # h_state (n_layers, batch, hidden_size)
        # rnn_out (batch, time_step, hidden_size)
        rnn_out, h_state = self.rnn(x, h_state)   # h_state是之前的隐层状态
        out = []
        for time in range(rnn_out.size(1)):
            every_time_out = rnn_out[:, time, :]       # 相当于获取每个时间点上的输出，然后过输出层
            out.append(self.output_layer(every_time_out))
        return torch.stack(out, dim=1), h_state       # torch.stack扩成[1, output_size, 1]


In [185]:

#定义递归树的节点信息
class TreeNode(object):
    def __init__(self,data,children=[],children_data=[]):
        self.parent = None
        self.children = children
        self.children_data = children_data
        self.data = data
        for child in children:
            child.parent = self

    def preorder(self, parent,res):
        if parent is None:
            return res
        if parent.children:
            res.append(torch.from_numpy(parent.data))
            for child in parent.children:
                self.preorder(child,res)
        return res
    
class RecursiveNetWork(object):
    def __init__(self,node_dim,child_count,activator,lr):
        self.node_dim = node_dim
        self.child_count = child_count
        self.lr = lr
        self.activator=activator
        self.W = np.ones((child_count * node_dim,node_dim))
        self.B = np.ones((node_dim,1))
        #生成最后的根节点
        self.root = None
    def concatenate(self,tree_nodes):
        concat = np.zeros((0,44))
        try:
            concat = np.concatenate((concat,tree_nodes[0].data))
        except:
            concat = np.concatenate((concat,tree_nodes[0].data.reshape(1,-1)))
        concat = np.concatenate((concat,tree_nodes[1].data))
        #for node in tree_nodes:
         #   cocat = np.concatenate((concat,node.data))
        return concat.reshape(-1,1)
    def forward(self,*children):
        #首先，将向量进行拼接
        children_data = self.concatenate(children)
        #前向传播，获取父节点的数据
        parent_data=IdentityActivator(np.dot(self.W.T,children_data)+ self.B)
        #生成根节点
        self.root = TreeNode(parent_data,children,children_data)
    def backward(self, parent_delta):
        self.calc_delta(parent_delta, self.root)
        self.W_grad, self.B_grad = self.calc_gradient(self.root)
    def calc_delta(self, parent_delta, parent):
        '''        计算δ值'''
        parent.delta = parent_delta
        if parent.children:
            children_delta = np.dot(self.W, parent_delta)
            slices = [(i, i * self.node_dim, (i + 1) * self.node_dim) for i in range(self.child_count)]
            # 针对每一个子节点，计算梯度
            for s in slices:
                self.calc_delta(children_delta[s[1]:s[2]], parent.children[s[0]])

    def calc_gradient(self, parent):
        '''      计算关于W和B的梯度的累加和'''
        W_grad = np.zeros((self.child_count * self.node_dim, self.node_dim))
        B_grad = np.zeros((self.node_dim, 1))
        if not parent.children:
            return W_grad, B_grad
        parent.W_grad = np.dot(parent.children_data, parent.delta.T)
        parent.B_grad = parent.delta
        W_grad += parent.W_grad
        B_grad += parent.B_grad
        for child in parent.children:
            W, B = self.calc_gradient(child)
            W_grad += W
            B_grad += B
        return W_grad, B_grad
    def update(self):
        self.W -= self.lr * self.W_grad
        self.B -= self.lr * self.B_grad

In [186]:
def train_rnn(train_dl,rnn1,rnn2,steps,gap,shape,save_path='C:/Aczh work place/3_paper/SCNRL_Master/baseline/rnn_model_2.pkl'):
    optimizer1 = torch.optim.SGD(rnn1.parameters(), lr=0.0001, momentum=0.2,weight_decay=0.001)
    data = torch.from_numpy(train_dl)
    regularization_loss,t = 0,len(train_dl)//steps
    loss_res=[]
    print('train_dl.shape is',train_dl.shape,'steps is',steps)
    for epoch in range(15):
        for step in range(steps-1):
            res=[1]
            print('.........................................')
            print('Now we strat to train epoch:',epoch,'step is',step)
            h_state = None  # 初始化隐藏层状态
            #h0 = model.init_hidden(DS.train_dl.shape[1])
            for id in range(step*t,(step+1)*t-gap-1,1):
                if id%100==0:
                    print(id/(t)*100,'%')
                    print('res[0]',res[0])
                rnn1.zero_grad()
                inputs = data[step*t+id:step*t+id+gap,:].reshape(-1,gap,shape[1])
                yhat, h_state = rnn1(inputs, h_state)
                h_state = h_state.detach()
                targets = data[step*t+id+1:step*t+(id+1)+gap,:].numpy()
                yhat_np=np.squeeze(yhat).detach().numpy()
                for epo in range(10):
                    for i in range(gap-1,-1,-1):
                        TH=TreeNode(yhat_np[i].reshape(1,-1),[],[])
                        if i==gap-1:
                            cd=TreeNode(targets[i].reshape(1,-1),[],[])
                            rnn2.forward(TH,cd)
                        else:
                            rnn2.forward(rnn2.root,TH)
                        gradient=np.ones((rnn2.node_dim,1),dtype=np.float64)
                        rnn2.backward(sensi_arr)
                res=[]
                if rnn2.root is not None:
                    res=rnn2.root.preorder(rnn2.root,res)
                print('res[0] is',res[0])
                tar_rel=torch.from_numpy(np.hstack(res).reshape(1,gap,-1))
                loss = L2_loss(yhat, tar_rel)
                loss.backward(retain_graph=True)
                optimizer1.step()
            print('Now the loss=',loss)
        loss_res.append(float(loss.detach().numpy()))
        if epoch>3 and (loss_res[epoch-1]-loss_res[epoch-2])<0.0001:
            try:
                torch.save(model, save_path)
                break
            except:
                print('Unsave model')
                break

rnn2=RecursiveNetWork(DS.dim,2,sigmoid,lr)
train_rnn(DS.train_dl,rnn1,rnn2,steps,gap,DS.shape)

train_dl.shape is (1400000, 44) steps is 200
.........................................
Now we strat to train epoch: 0 step is 0
0.0 %
res[0] 1
res[0] is tensor([[4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.0111e+16],
        [4.

res[0] is tensor([[4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16],
        [4.4193e+16]], dtype=torch.float64)


res[0] is tensor([[4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16],
        [4.4768e+16]], dtype=torch.float64)


res[0] is tensor([[4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16],
        [4.3291e+16]], dtype=torch.float64)


KeyboardInterrupt: 

In [177]:
    num_model=8
    steps,gap=200,10
    data_path='C:/Aczh work place/3_paper/SNCRL_Dataset/CellCycle/'
    DS=Dataset(data_path,1)
    print('train_dl.shape',DS.train_dl.shape)
    print('test_dl.shape',DS.test_dl.shape)
    print('ori_data.shape',DS.ori_data.shape)
    #print('len(Ds)',DS.ori_data)
    print('len(Ds)',DS.ori_data[:len(DS.ori_data),13:20].shape)
    print('ori_bound is',DS.ori_upbound,DS.ori_lowbound)
    #save_path='C:/Aczh work place/3_paper/SCNRL_Master/model/CellCycle/'
    save_path='C:/Aczh work place/3_paper/SCNRL_Master/baseline/rnn_model_2.pkl'
    embedding_dim=DS.dim
    hidden_dim=DS.dim*3
    voacb_size=1
    batch_size=1
    target_size=DS.dim
    num_layers = 2
    input_size=DS.dim
    hidden_size=DS.dim*3
    lr=0.001
    #model_MLP=MLP(DS.dim+1)
    #model=LSTM(embedding_dim,batch_size,hidden_dim,voacb_size,target_size,num_layers)
    rnn1 = RNN(input_size, hidden_size, num_layers, target_size)
    rnn2=RecursiveNetWork(DS.dim,gap,sigmoid,lr)


successly load time


KeyboardInterrupt: 